In [23]:
import pandas as pd
import numpy as np
import random

In [36]:
# ysi_mp_csv_path = '/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/CP-008 Session 25 - Pre-Pivotal2/all_data/prospective_8S17_v14h_2022_5day_AllRef/performance/ysi including 19c 20 22.csv'
ysi_mp_csv_path = '/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/CP-008 Session 25 - Pre-Pivotal2/experimental/prospective_8S20_v15_fault_dt_2022_5day_dayOneEnabled_thresh0.5_v2_AllRef/performance/ysi including 19c 20 22.csv'
ysi_data = pd.read_csv(ysi_mp_csv_path)

In [45]:
chem_list = ['2.5.11-PPD+KCL', 'Dynamic-Thick', 'Dynamic-Thin', 'Static-Thin']
tier_list = ['Tier 1: 3/3 Clinical Use']
# group_list = ['Pre-Piv-2','eBlinq19c', 'eBlinq20', 'eBlinq22']
# group_list = ['Pre-Piv-2']
group_list = ['eBlinq19c', 'eBlinq20', 'eBlinq22']

In [46]:
valid_index = ysi_data[(ysi_data['Chemistry'].isin(chem_list)) & (ysi_data['Sorting Tier'].isin(tier_list)) & (ysi_data['Grouping'].isin(group_list))].index

In [47]:
ysi_data_valid = ysi_data.loc[valid_index].copy()

In [48]:
seed_of_seed = 1
seed_range = 100000
num_bootstrap = 999

sid_df = pd.DataFrame({'sensor_id': pd.unique(ysi_data_valid['sensor_id'])})

all_seeds = list(range(1,seed_range))
random.seed(seed_of_seed)
random.shuffle(all_seeds)
seeds_for_bootstrap = all_seeds[0:num_bootstrap]
mard_by_bootstrap = np.full(num_bootstrap,np.nan)
for seedIdx,iSeed in enumerate(seeds_for_bootstrap):      
    bootstrap_df = sid_df.sample(frac=1, replace=True, random_state=iSeed)
    bootstrap_df = bootstrap_df.merge(ysi_data_valid,how='left')
    mard_by_bootstrap[seedIdx] = bootstrap_df['ard'].mean()

print(np.percentile(mard_by_bootstrap,2.5))
print(np.percentile(mard_by_bootstrap,97.5))

14.40221940100136
18.50550183087174
